### Imports

In [1]:
# Classes
from D_S_Actor import D_S_Actor
from D_S_Critic import D_S_Critic
from A_R_Actor import A_R_Actor
from P_Q_Actor import P_Q_Actor
from A_R_P_Q_Critic import A_R_P_Q_Critic
from ReplayBuffer import ReplayBuffer
from ReplayBuffer_A_R_P_Q import ReplayBuffer_A_R_P_Q

Using TensorFlow backend.


In [2]:
# Libraries
import tensorflow as tf
import collections
import random
import numpy as np
import math
import copy

### Renewable and Demand Functions

In [3]:
# Functions
def get_renewable(index, time):
    lamb = [[2.667e-07, 0.541, 6.5965, 4.3712], [8.8281, 10.2997, 9.8301, 9.7514], [8.8281, 10.2997, 9.8301, 9.7514]]
    energy = np.random.poisson(lam=lamb[index][time], size=1)
    energy = min([8, energy])  # clipping the value so that it can't exceede 8
    energy = int(math.floor(energy))
    return energy


# Write this function
def get_demand(index, time):
    non_adl = [3, 4, 5, 6]
    prob_non_adl = [[0.4, 0.3, 0.2, 0.1], [0.1, 0.4, 0.3, 0.2], [0.1, 0.3, 0.4, 0.2], [0.2, 0.3, 0.1, 0.4]]
    demand = np.random.choice(non_adl, p=prob_non_adl[time - 1])
    return int(demand)

In [4]:
# Testing
for i in range(3):
    print('Renewable',i,':',get_renewable(i, 0))
    print('Demand',i,':', get_demand(i, 0))
    print('---------------------')

Renewable 0 : 0
Demand 0 : 6
---------------------
Renewable 1 : 8
Demand 1 : 6
---------------------
Renewable 2 : 8
Demand 2 : 6
---------------------


### Initializations

In [5]:
# Constants and Lists
no_of_agents = 3
k = 5
grid_price = 20
lower_price = grid_price - 5
names = ['Agent_1', 'Agent_2', 'Agent_3']  # Names for the agents
agent = []
total_no_of_iterations = 1000000
time_steps_per_day = 4
no_of_sub_transactions = 0
c1 = 20

In [6]:
#  'ADL_Actor': ADL_Actor(), 'ADL_Critic': ADL_Critic(), Add this when ADL is required
for i in range(no_of_agents):
    agent.append({'index': i, 'name': names[i],
                  'D_S_Actor': D_S_Actor(5, 0.01, 0.1), 'D_S_Critic': D_S_Critic(5, 1),
                  'A_R_Actor': A_R_Actor((3 + 3*no_of_agents),0.01, 0.1, no_of_agents), 'P_Q_Actor': P_Q_Actor((3 + 4*no_of_agents),0.01, 0.1, no_of_agents), 'A_R_P_Q_Critic': A_R_P_Q_Critic((3 + 3*no_of_agents), (3 + 4*no_of_agents), no_of_agents, no_of_agents, no_of_agents),
                  'Battery': 3, 'Renewable': 0,
                  'Demand': 0, 'ADL_Value': 0, 'ADL_State': [], 'ADL_Action': [], 'D_S_State': [],
                  'D_S_Action': [], 'Current_Demand': 0, 'Main_Reward': 0,
                  'A_R_State': [], 'A_R_Action': [], 'P_Q_State': [], 'P_Q_Action': [], 'Via_Matrix': 0,
                  'Sub_Reward': 0,
                  'A_R_Vector': np.zeros(no_of_agents), 'P_Vector': np.zeros(no_of_agents),
                  'Q_Vector': np.zeros(no_of_agents), 'Status': [], 'Customised_Status': [],
                  'D_S_Buffer': ReplayBuffer(1), 'A_R_P_Q_Buffer': ReplayBuffer_A_R_P_Q(no_of_agents, no_of_agents, no_of_agents, 1)})

E:\Chanakya\Markets in Smartgrids\Config\Debugging\A_R_Actor.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  self.model = models.Model(input=inputs, output=outputs)
E:\Chanakya\Markets in Smartgrids\Config\Debugging\P_Q_Actor.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  self.model = models.Model(input=inputs, output=[output_p, output_q, output_q_traded])
E:\Chanakya\Markets in Smartgrids\Config\Debugging\A_R_Actor.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  self.model = models.Model(input=inputs, output=outputs)
E:\Chanakya\Markets in Smartgrids\Config\Debugging\P_Q_Actor.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  self.model = models.Model(input=inputs, output=[output_p, output_q, output_q_t

In [7]:
# Testing
print(agent[0])

{'index': 0, 'name': 'Agent_1', 'D_S_Actor': <D_S_Actor.D_S_Actor object at 0x00000290287997F0>, 'D_S_Critic': <D_S_Critic.D_S_Critic object at 0x0000029028882E48>, 'A_R_Actor': <A_R_Actor.A_R_Actor object at 0x0000029029ADD6A0>, 'P_Q_Actor': <P_Q_Actor.P_Q_Actor object at 0x0000029029DCD908>, 'A_R_P_Q_Critic': <A_R_P_Q_Critic.A_R_P_Q_Critic object at 0x000002902B012390>, 'Battery': 3, 'Renewable': 0, 'Demand': 0, 'ADL_Value': 0, 'ADL_State': [], 'ADL_Action': [], 'D_S_State': [], 'D_S_Action': [], 'Current_Demand': 0, 'Main_Reward': 0, 'A_R_State': [], 'A_R_Action': [], 'P_Q_State': [], 'P_Q_Action': [], 'Via_Matrix': 0, 'Sub_Reward': 0, 'A_R_Vector': array([0., 0., 0.]), 'P_Vector': array([0., 0., 0.]), 'Q_Vector': array([0., 0., 0.]), 'Status': [], 'Customised_Status': [], 'D_S_Buffer': <ReplayBuffer.ReplayBuffer object at 0x000002902B2D42E8>, 'A_R_P_Q_Buffer': <ReplayBuffer_A_R_P_Q.ReplayBuffer_A_R_P_Q object at 0x000002902B2D4978>}


### Getting Demand, Status Vectors etc

In [8]:
buyers = []
sellers = []
nothing = []
status = []  # Whether the agents are buyers or sellers or nothing (1: Sellers, 0:Buyers, -1: Nothing)

j = 0 # current time is 0
# Getting the D_S_State and storing it in respective variables and replay buffers
for i in range(no_of_agents):

    agent[i]['D_S_State'] = []
    agent[i]['D_S_Action'] = 0
    agent[i]['Main_Reward'] = 0
    agent[i]['Sub_Reward'] = 0
    agent[i]['Renewable'] = get_renewable(i, j % time_steps_per_day)
    agent[i]['Demand'] = get_demand(i, j % time_steps_per_day)
    agent[i]['D_S_State'] = np.asarray([[j % time_steps_per_day, agent[i]['Battery'], agent[i]['Renewable'],
                             agent[i]['Demand'], grid_price]])
    agent[i]['D_S_Buffer'].store_transition_state(agent[i]['D_S_State'])

In [9]:
### Testing 
print('Agent 0 Summary:')
print(agent[0])
print('-----------------------------------------------------------------------------------------------------------------------')
print('D_S_State Replay Buffer:')
print(agent[0]['D_S_Buffer'].states[0])

Agent 0 Summary:
{'index': 0, 'name': 'Agent_1', 'D_S_Actor': <D_S_Actor.D_S_Actor object at 0x00000290287997F0>, 'D_S_Critic': <D_S_Critic.D_S_Critic object at 0x0000029028882E48>, 'A_R_Actor': <A_R_Actor.A_R_Actor object at 0x0000029029ADD6A0>, 'P_Q_Actor': <P_Q_Actor.P_Q_Actor object at 0x0000029029DCD908>, 'A_R_P_Q_Critic': <A_R_P_Q_Critic.A_R_P_Q_Critic object at 0x000002902B012390>, 'Battery': 3, 'Renewable': 0, 'Demand': 4, 'ADL_Value': 0, 'ADL_State': [], 'ADL_Action': [], 'D_S_State': array([[ 0,  3,  0,  4, 20]]), 'D_S_Action': 0, 'Current_Demand': 0, 'Main_Reward': 0, 'A_R_State': [], 'A_R_Action': [], 'P_Q_State': [], 'P_Q_Action': [], 'Via_Matrix': 0, 'Sub_Reward': 0, 'A_R_Vector': array([0., 0., 0.]), 'P_Vector': array([0., 0., 0.]), 'Q_Vector': array([0., 0., 0.]), 'Status': [], 'Customised_Status': [], 'D_S_Buffer': <ReplayBuffer.ReplayBuffer object at 0x000002902B2D42E8>, 'A_R_P_Q_Buffer': <ReplayBuffer_A_R_P_Q.ReplayBuffer_A_R_P_Q object at 0x000002902B2D4978>}
------

#### Getting D_S_Actions and Storing it, Customized Status and Sellers and Buyers

In [10]:
# Getting the D_S_Actions and storing it in respective variables and replay buffers using the constraints
for i in range(no_of_agents):
    
    agent[i]['D_S_Action'] = agent[i]['D_S_Actor'].action(agent[i]['D_S_State'])
    if agent[i]['D_S_Action'][0][0] > 0.999999999:
        agent[i]['Current_Demand'] = (np.arctanh(0.999999999) + 18.71497387) * \
                             (agent[i]['Battery'] + agent[i]['Demand']) / 37.42994775 + agent[i]['Renewable'] - agent[i]['Demand']
    elif agent[i]['D_S_Action'][0][0] < - 0.999999999:
        agent[i]['Current_Demand'] = (np.arctanh(-0.99999999) + 18.71497387) * \
                             (agent[i]['Battery'] + agent[i]['Demand']) / 37.42994775 + agent[i]['Renewable'] - agent[i]['Demand']
    else:   
        agent[i]['Current_Demand'] = (np.arctanh(agent[i]['D_S_Action'][0][0]) + 18.71497387) * \
                                 (agent[i]['Battery'] + agent[i]['Demand']) / 37.42994775 + agent[i]['Renewable'] - agent[i]['Demand']
    
    agent[i]['D_S_Buffer'].store_transition_action(agent[i]['D_S_Action'])
    agent[i]['Battery'] = max(0, (agent[i]['Renewable'] + agent[i]['Battery'] - agent[i]['Current_Demand'] - agent[i]['Demand']))
    if agent[i]['Current_Demand'] < 0:
        status.append(0)
    elif agent[i]['Current_Demand'] == 0:
        status.append(-1)
    else:
        status.append(1)

In [11]:
### Testing
for i in range(no_of_agents):
    print('Agent:', i, '| Demand:', agent[i]['Demand'], '| Renewable:', agent[i]['Renewable'],'| D_S_Action:', agent[i]['D_S_Actor'].action(agent[i]['D_S_State']), '| D_S_Action_Value:', agent[i]['Current_Demand'], '| Battery:', agent[i]['Battery'])
print('-------------------------------------------------')
print('Status:', status)

Agent: 0 | Demand: 4 | Renewable: 0 | D_S_Action: [[0.56378144]] | D_S_Action_Value: -0.38061657019192285 | Battery: 0
Agent: 1 | Demand: 4 | Renewable: 8 | D_S_Action: [[0.99170774]] | D_S_Action_Value: 8.01255717993055 | Battery: 0
Agent: 2 | Demand: 5 | Renewable: 8 | D_S_Action: [[-0.5356195]] | D_S_Action_Value: 6.872189524988791 | Battery: 0
-------------------------------------------------
Status: [0, 1, 1]


In [12]:
# Customized Status
for i in range(no_of_agents):
    agent[i]['Status'] = status
    status_1 = status
    if agent[i]['Current_Demand'] < 0:
        for l in range(len(status)):
            if status_1[i] == (-1):
                status_1[i] = 0
    elif agent[i]['Current_Demand'] == 0:
        pass
    else:
        for l in range(len(status)):
            if status_1[i] == (-1):
                status_1[i] = 1
    agent[i]['Customised_Status'] = np.array(status_1)

In [13]:
### Testing 
for i in range(no_of_agents):
     print('Agent:', i, '| Current_Demand:', agent[i]['Current_Demand'], '| Customized_Status:',  agent[i]['Customised_Status']) 

Agent: 0 | Current_Demand: -0.38061657019192285 | Customized_Status: [0 1 1]
Agent: 1 | Current_Demand: 8.01255717993055 | Customized_Status: [0 1 1]
Agent: 2 | Current_Demand: 6.872189524988791 | Customized_Status: [0 1 1]


In [14]:
# Separating into Buyers, Sellers or Nothing and updating the Via_Matrix
for i in range(no_of_agents):
    if agent[i]['Current_Demand'] < 0:
        agent[i]['Via_Matrix'] = np.diag(agent[i]['Customised_Status'])
        buyers.append(agent[i])
    elif agent[i]['Current_Demand'] == 0:
        agent[i]['Via_Matrix'] = np.diag(agent[i]['Customised_Status'])
        nothing.append(agent[i])
    else:
        agent[i]['Via_Matrix'] = np.eye(len(agent[i]['Customised_Status'])) - np.diag(
            agent[i]['Customised_Status'])
        agent[i]['Customised_Status'] = 1 - np.array(agent[i]['Customised_Status'])  
        sellers.append(agent[i])

In [15]:
### Testing Sellers and Buyers
for i in range(len(buyers)):
    print('Buyer:', buyers[i]['index'], '| Current_Demand:', buyers[i]['Current_Demand'], '| Customized_Status:',  buyers[i]['Customised_Status'], '| Via Matrix:', buyers[i]['Via_Matrix'])
for i in range(len(sellers)):
    print('Seller:',sellers[i]['index'] , '| Current_Demand:', sellers[i]['Current_Demand'], '| Customized_Status:',  sellers[i]['Customised_Status'], '| Via Matrix:', sellers[i]['Via_Matrix'])
        

Buyer: 0 | Current_Demand: -0.38061657019192285 | Customized_Status: [0 1 1] | Via Matrix: [[0 0 0]
 [0 1 0]
 [0 0 1]]
Seller: 1 | Current_Demand: 8.01255717993055 | Customized_Status: [1 0 0] | Via Matrix: [[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Seller: 2 | Current_Demand: 6.872189524988791 | Customized_Status: [1 0 0] | Via Matrix: [[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


### Sub Transactions

In [16]:
# Storing in the buffer
for m in range(len(sellers)):
    sellers[m]['A_R_P_Q_Buffer'].store_transition_via_matrix(sellers[m]['Via_Matrix'])
    sellers[m]['A_R_P_Q_Buffer'].store_transition_customized_status(sellers[m]['Customised_Status'])
for m in range(len(buyers)):
    buyers[m]['A_R_P_Q_Buffer'].store_transition_via_matrix(buyers[m]['Via_Matrix'])
    buyers[m]['A_R_P_Q_Buffer'].store_transition_customized_status(buyers[m]['Customised_Status'])

#### A_R_Actions Sellers (Asumming its the first time step for the sellers)

In [17]:
# A_R for Sellers
# If the Sub-Transaction time is 0, Ignore the A_R_Network, the action would be a zero everywhere
k = 0 # for the first time step
if k == 0:
    for m in range(len(sellers)):
        sellers[m]['P_Q_State'] = [k, sellers[m]['Demand'], sellers[m]['Current_Demand']]
        sellers[m]['P_Q_State'].extend(sellers[m]['Customised_Status'])
        sellers[m]['P_Q_State'].extend(sellers[m]['P_Vector'])
        sellers[m]['P_Q_State'].extend(sellers[m]['Q_Vector'])
        sellers[m]['A_R_State'] = sellers[m]['P_Q_State']

        sellers[m]['A_R_P_Q_Buffer'].store_transition_state_a_r(sellers[m]['P_Q_State'])
        sellers[m]['A_R_P_Q_Buffer'].store_transition_action_a_r(np.zeros(no_of_agents))

        sellers[m]['P_Q_State'].extend(np.zeros(no_of_agents))
        sellers[m]['A_R_P_Q_Buffer'].store_transition_state_p_q(sellers[m]['P_Q_State'])
        sellers[m]['P_Q_State'] = np.asarray([sellers[m]['P_Q_State']])  

In [18]:
### Testing 
for m in range(len(sellers)):
    print('Sellers:', sellers[m]['index'], '| A_R_State:', sellers[m]['A_R_State'], '| P_Q_State:', sellers[m]['P_Q_State'])

Sellers: 1 | A_R_State: [0, 4, 8.01255717993055, 1, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | P_Q_State: [[0.         4.         8.01255718 1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]]
Sellers: 2 | A_R_State: [0, 5, 6.872189524988791, 1, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | P_Q_State: [[0.         5.         6.87218952 1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]]


#### P_Q_Action for the Sellers

In [19]:
for m in range((len(sellers))):
    action_p, action_q, action_q_traded = sellers[m]['P_Q_Actor'].action(sellers[m]['P_Q_State'])
    print('Action P:', action_p, '| Action Q:', action_q, '| Action Q_Traded:', action_q_traded)

    action_p = np.matmul(sellers[m]['Via_Matrix'], action_p[0])
    action_q = np.matmul(sellers[m]['Via_Matrix'], action_q[0]) / np.matmul(
        np.matmul(sellers[m]['Via_Matrix'], action_q[0]), sellers[m]['Customised_Status'])
    action_q_traded = action_q_traded[0]
    print('Action P:', action_p, '| Action Q:', action_q, '| Action Q_Traded:', action_q_traded)


    sellers[m]['A_R_P_Q_Buffer'].store_transition_action_p_q(action_p, action_q, action_q_traded)
    action_p = ((np.arctanh(action_p) + 18.714973875118524) / 37.42994775023705) * (grid_price - lower_price) + lower_price
    action_q = ((np.arctanh(action_q_traded) + 18.714973875118524) / 37.42994775023705) * action_q * \
                sellers[m]['Current_Demand']
    action_p = np.matmul(sellers[m]['Via_Matrix'], action_p)
    action_q = np.matmul(sellers[m]['Via_Matrix'], action_q)
    print('Action_Q_Traded_Value', action_q)
    print('------------------------------------------------')
    for n in range(len(buyers)):
        buyers[n]['P_Vector'][sellers[m]['index']] = action_p[buyers[n]['index']]
        buyers[n]['Q_Vector'][sellers[m]['index']] = action_q[buyers[n]['index']]

Action P: [[-0.9597747   0.46511024 -0.9778304 ]] | Action Q: [[0.17435935 0.00447799 0.8211626 ]] | Action Q_Traded: [[-0.99992037]]
Action P: [-0.95977467  0.          0.        ] | Action Q: [1. 0. 0.] | Action Q_Traded: [-0.99992037]
Action_Q_Traded_Value [2.92189488 0.         0.        ]
------------------------------------------------
Action P: [[-0.9998849   0.65140986  0.8648633 ]] | Action Q: [[0.19835323 0.794137   0.00750978]] | Action Q_Traded: [[-0.82434446]]
Action P: [-0.9998849  0.         0.       ] | Action Q: [1. 0. 0.] | Action Q_Traded: [-0.82434446]
Action_Q_Traded_Value [3.22123992 0.         0.        ]
------------------------------------------------


In [20]:
### Testing
for n in range(len(buyers)):
    print('P_Vector:', buyers[n]['P_Vector'])
    print('Q_Vector:', buyers[n]['Q_Vector'])
    print('---------------------------------')

P_Vector: [ 0.         17.2404426  16.84792664]
Q_Vector: [0.         2.92189488 3.22123992]
---------------------------------


#### A_R_Action of the buyers

In [21]:
# A_R_Action for Buyers
for m in range(len(buyers)):
    buyers[m]['A_R_State'] = [k, buyers[m]['Demand'], buyers[m]['Current_Demand']]
    buyers[m]['A_R_State'].extend(buyers[m]['Customised_Status'])
    buyers[m]['A_R_State'].extend(buyers[m]['P_Vector'])
    buyers[m]['A_R_State'].extend(buyers[m]['Q_Vector'])
    print('A_R_State:', buyers[m]['A_R_State'])

    buyers[m]['A_R_P_Q_Buffer'].store_transition_state_a_r(buyers[m]['A_R_State'])
    buyers[m]['A_R_State'] = np.asarray([buyers[m]['A_R_State']])
    
    buyers[m]['A_R_Action'] = buyers[m]['A_R_Actor'].action(buyers[m]['A_R_State'])
    print('A_R_Action:', buyers[m]['A_R_Action'])
    buyers[m]['A_R_Action'] = np.matmul(buyers[m]['Via_Matrix'], buyers[m]['A_R_Action'][0])
    print('A_R_Action:', buyers[m]['A_R_Action'])
    buyers[m]['A_R_Vector'] = np.asarray(buyers[m]['A_R_Action']) > 0.5
    print('A_R_Vector:', buyers[m]['A_R_Vector'] )

    buyers[m]['P_Q_State'] = list(buyers[m]['A_R_State'][0])
    buyers[m]['P_Q_State'].extend(buyers[m]['A_R_Action'])
    print('P_Q_State:', buyers[m]['P_Q_State'])
    print('---------------------------------------------')

    buyers[m]['A_R_P_Q_Buffer'].store_transition_action_a_r(buyers[m]['A_R_Action'])
    buyers[m]['A_R_P_Q_Buffer'].store_transition_state_p_q(buyers[m]['P_Q_State'])
    buyers[m]['P_Q_State'] = np.asarray([buyers[m]['P_Q_State']])

    total_q = 0
    reward = 0
    
    
    for n in range(len(sellers)):
        # The P and Q vector of the buyers have to be the P and Q values of each seller. That means that
        # the  each P would be positive and each Q would be positive
        sellers[n]['Sub_Reward'] = sellers[n]['Sub_Reward'] + buyers[m]['Q_Vector'][sellers[n]['index']] * \
                                   buyers[m]['P_Vector'][sellers[n]['index']] * \
                                   buyers[m]['A_R_Vector'][sellers[n]['index']]
        print('Sellers:', sellers[n]['index'], '| Sub Reward:', sellers[n]['Sub_Reward'], '| Current_Demand:', sellers[n]['Current_Demand'])

        # The negative sign makes sense as the values of the Q_vector are positive and the current demand
        # is positive
        sellers[n]['Current_Demand'] = max(0, sellers[n]['Current_Demand'] - buyers[m]['Q_Vector'][
            sellers[n]['index']] * buyers[m]['A_R_Vector'][sellers[n]['index']])
        print('Sellers:', sellers[n]['index'], '| Sub Reward:', sellers[n]['Sub_Reward'], '| Current_Demand:', sellers[n]['Current_Demand'])

        print('--------------------------------------------------------------------------')
        # The positive sign makes sense as the values of the Q_vector are positive
        total_q = total_q + buyers[m]['Q_Vector'][sellers[n]['index']] * buyers[m]['A_R_Vector'][
            sellers[n]['index']]

        # The negative sign makes sense as the values of the Q_vector are positive
        reward = reward - buyers[m]['Q_Vector'][sellers[n]['index']] * buyers[m]['P_Vector'][
            sellers[n]['index']] * \
                 buyers[m]['A_R_Vector'][sellers[n]['index']]
    
    print('Buyers:', buyers[m]['index'], '| Sub_Reward:', buyers[m]['Sub_Reward'], '| Current Demand:', buyers[m]['Current_Demand'])
    if (-total_q) < buyers[m]['Current_Demand']:
        # buyers[m]['Current_Demand'] is negative and total_q is positive. Reward is increased as the excess
        # energy is sold back to the main grid.
        reward = reward + (total_q + buyers[m]['Current_Demand']) * lower_price
        buyers[m]['Sub_Reward'] = buyers[m]['Sub_Reward'] + reward
        buyers[m]['Current_Demand'] = 0

    else:
        buyers[m]['Sub_Reward'] = buyers[m]['Sub_Reward'] + reward
        buyers[m]['Current_Demand'] = buyers[m]['Current_Demand'] + total_q
    
    print('Buyers:', buyers[m]['index'], '| Sub_Reward:', buyers[m]['Sub_Reward'], '| Current Demand:', buyers[m]['Current_Demand'])

A_R_State: [0, 4, -0.38061657019192285, 0, 1, 1, 0.0, 17.24044260051503, 16.84792664240576, 0.0, 2.9218948788154964, 3.2212399179571607]
A_R_Action: [[0.9139784  0.98560035 0.65795666]]
A_R_Action: [0.         0.98560035 0.65795666]
A_R_Vector: [False  True  True]
P_Q_State: [0.0, 4.0, -0.38061657019192285, 0.0, 1.0, 1.0, 0.0, 17.24044260051503, 16.84792664240576, 0.0, 2.9218948788154964, 3.2212399179571607, 0.0, 0.9856003522872925, 0.6579566597938538]
---------------------------------------------
Sellers: 1 | Sub Reward: 50.37476094295739 | Current_Demand: 8.01255717993055
Sellers: 1 | Sub Reward: 50.37476094295739 | Current_Demand: 5.090662301115053
--------------------------------------------------------------------------
Sellers: 2 | Sub Reward: 54.271213835331395 | Current_Demand: 6.872189524988791
Sellers: 2 | Sub Reward: 54.271213835331395 | Current_Demand: 3.65094960703163
--------------------------------------------------------------------------
Buyers: 0 | Sub_Reward: 0 | Cur

#### P_Q_Action of the Buyers

In [22]:
buyers[0]['Current_Demand'] = -2 # Synthetic Demand
for m in range(len(buyers)):

    if k == (no_of_sub_transactions - 1):
        buyers[m]['A_R_P_Q_Buffer'].store_transition_action_p_q(np.zeros(no_of_agents),
                                                                np.zeros(no_of_agents), np.zeros(1))

    else:
        action_p, action_q, action_q_traded = buyers[m]['P_Q_Actor'].action(buyers[m]['P_Q_State'])

    action_p = np.matmul(buyers[m]['Via_Matrix'], action_p[0])
    action_q = np.matmul(buyers[m]['Via_Matrix'], action_q[0]) / np.matmul(
        np.matmul(buyers[m]['Via_Matrix'], action_q[0]), buyers[m]['Customised_Status'])
    action_q_traded = action_q_traded[0]
    print('Action P:', action_p, '| Action Q:', action_q, '| Action Q_Traded:', action_q_traded)

    buyers[m]['A_R_P_Q_Buffer'].store_transition_action_p_q(action_p, action_q, action_q_traded)
    action_p = ((np.arctanh(action_p) + 18.714973875118524) / 37.42994775023705) * (grid_price - lower_price) + lower_price
    action_q = -((np.arctanh(action_q_traded) + 18.714973875118524) / 37.42994775023705) * action_q * \
               buyers[m]['Current_Demand'] + action_q * buyers[m]['Current_Demand']
    action_p = np.matmul(buyers[m]['Via_Matrix'], action_p)
    action_q = np.matmul(buyers[m]['Via_Matrix'], action_q)
    
    print('Action P:', action_p, '| Action Q:', action_q, '| Action Q_Traded:', action_q_traded)
    print('Action_Q_Traded_Value', action_q)
    print('------------------------------------------------')

    for n in range(len(sellers)):
        sellers[n]['P_Vector'][buyers[m]['index']] = action_p[sellers[n]['index']]
        sellers[n]['Q_Vector'][buyers[m]['index']] = action_q[sellers[n]['index']]

Action P: [ 0.         -0.99626505  0.99967837] | Action Q: [0.         0.59210381 0.40789619] | Action Q_Traded: [-0.9998122]
Action P: [ 0.         17.08046296 18.08343025] | Action Q: [ 0.         -0.73879502 -0.50895074] | Action Q_Traded: [-0.9998122]
Action_Q_Traded_Value [ 0.         -0.73879502 -0.50895074]
------------------------------------------------


In [23]:
### Testing
for n in range(len(sellers)):
    print('P_Vector:', sellers[n]['P_Vector'])
    print('Q_Vector:', sellers[n]['Q_Vector'])
    print('---------------------------------')

P_Vector: [17.08046296  0.          0.        ]
Q_Vector: [-0.73879502  0.          0.        ]
---------------------------------
P_Vector: [18.08343025  0.          0.        ]
Q_Vector: [-0.50895074  0.          0.        ]
---------------------------------


#### A_R_Action of the Sellers for any other tine step

In [24]:
for m in range(len(sellers)):
    sellers[m]['A_R_State'] = [k, sellers[m]['Demand'], sellers[m]['Current_Demand']]
    sellers[m]['A_R_State'].extend(sellers[m]['Customised_Status'])
    sellers[m]['A_R_State'].extend(sellers[m]['P_Vector'])
    sellers[m]['A_R_State'].extend(sellers[m]['Q_Vector'])
    print('A_R_State:', sellers[m]['A_R_State'])

    sellers[m]['A_R_P_Q_Buffer'].store_transition_state_a_r(sellers[m]['A_R_State'])
    sellers[m]['A_R_State'] = np.asarray([sellers[m]['A_R_State']])

    sellers[m]['A_R_Action'] = sellers[m]['A_R_Actor'].action(sellers[m]['A_R_State'])
    print('A_R_Action:', sellers[m]['A_R_Action'])
    sellers[m]['A_R_Action'] = np.matmul(sellers[m]['Via_Matrix'], sellers[m]['A_R_Action'][0])
    print('A_R_Action:', sellers[m]['A_R_Action'])
    sellers[m]['A_R_Vector'] = np.asarray(sellers[m]['A_R_Action']) > 0.5
    print('A_R_Vector:', sellers[m]['A_R_Vector'])

    sellers[m]['P_Q_State'] = list(sellers[m]['A_R_State'][0])
    sellers[m]['P_Q_State'].extend(sellers[m]['A_R_Action'])

    sellers[m]['A_R_P_Q_Buffer'].store_transition_action_a_r(sellers[m]['A_R_Action'])
    sellers[m]['A_R_P_Q_Buffer'].store_transition_state_p_q(sellers[m]['P_Q_State'])
    print('P_Q_State:', sellers[m]['P_Q_State'])
    print('---------------------------------------------')
    
    total_q = 0
    reward = 0
    
    for n in range(len(buyers)):
        # The P and Q vector of the Sellers have to be the P and Q values of each buyer. That means that
        # the  each P would be positive and each Q would be negative
        buyers[n]['Sub_Reward'] = buyers[n]['Sub_Reward'] + sellers[m]['Q_Vector'][buyers[n]['index']] * \
                                  sellers[m]['P_Vector'][buyers[n]['index']] * \
                                  sellers[m]['A_R_Vector'][buyers[n]['index']]
        print('Buyers:', buyers[n]['index'], '| Sub Reward:', buyers[n]['Sub_Reward'], '| Current_Demand:', buyers[n]['Current_Demand'])

        # The negative sign makes sense as the values of the Q_vector are negative and the current demand
        # is positive
        buyers[n]['Current_Demand'] = min(0, buyers[n]['Current_Demand'] - sellers[m]['Q_Vector'][
            buyers[n]['index']] * sellers[m]['A_R_Vector'][buyers[n]['index']])
        print('Buyers:', buyers[n]['index'], '| Sub Reward:', buyers[n]['Sub_Reward'], '| Current_Demand:', buyers[n]['Current_Demand'])
        
        print('--------------------------------------------------------------------')

        # The negative sign makes sense as the values of the Q_vector are negative
        total_q = total_q - sellers[m]['Q_Vector'][buyers[n]['index']] * sellers[m]['A_R_Vector'][buyers[n]['index']]

        # The negative sign makes sense as the values of the Q_vector are negative
        reward = reward - sellers[m]['Q_Vector'][buyers[n]['index']] * sellers[m]['P_Vector'][
            buyers[n]['index']] * sellers[m]['A_R_Vector'][buyers[n]['index']]
    
    print('Sellers:', sellers[m]['index'], '| Sub_Reward:', sellers[m]['Sub_Reward'], '| Current Demand:', sellers[m]['Current_Demand'])
    if total_q > sellers[m]['Current_Demand']:
        # The reward is decreased as the excess energy sold is bought from the main grid.
        reward = reward - (total_q - sellers[m]['Current_Demand']) * grid_price
        sellers[m]['Sub_Reward'] = sellers[m]['Sub_Reward'] + reward
        sellers[m]['Current_Demand'] = 0

    else:
        sellers[m]['Sub_Reward'] = sellers[m]['Sub_Reward'] + reward
        sellers[m]['Current_Demand'] = sellers[m]['Current_Demand'] - total_q
    print('Sellers:', sellers[m]['index'], '| Sub_Reward:', sellers[m]['Sub_Reward'], '| Current Demand:', sellers[m]['Current_Demand'])
    print('-----------------------------------------------------------------------------------------------------------------------------')

A_R_State: [0, 4, 5.090662301115053, 1, 0, 0, 17.080462958791767, 0.0, 0.0, -0.7387950163096526, 0.0, 0.0]
A_R_Action: [[0.5937217  0.20573992 0.00752104]]
A_R_Action: [0.59372169 0.         0.        ]
A_R_Vector: [ True False False]
P_Q_State: [0.0, 4.0, 5.090662301115053, 1.0, 0.0, 0.0, 17.080462958791767, 0.0, 0.0, -0.7387950163096526, 0.0, 0.0, 0.5937216877937317, 0.0, 0.0]
---------------------------------------------
Buyers: 0 | Sub Reward: -30.827162289794753 | Current_Demand: -2
Buyers: 0 | Sub Reward: -30.827162289794753 | Current_Demand: -1.2612049836903474
--------------------------------------------------------------------
Sellers: 1 | Sub_Reward: 50.37476094295739 | Current Demand: 5.090662301115053
Sellers: 1 | Sub_Reward: 62.993721853174364 | Current Demand: 4.3518672848054
-----------------------------------------------------------------------------------------------------------------------------
A_R_State: [0, 5, 3.65094960703163, 1, 0, 0, 18.083430247398102, 0.0, 0.0